In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

import tensorflow as tf

import yfinance as yf

2025-10-02 18:13:40.817455: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-02 18:13:40.817892: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 18:13:40.876386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-02 18:13:42.586843: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

# Loading the data

Financials

In [2]:
balance_sheet = pd.read_csv('data/financials/AAPL_balance_sheet.csv', header=None)
capital_gains = pd.read_csv('data/financials/AAPL_capital_gains.csv', header=None)
cash_flow = pd.read_csv('data/financials/AAPL_cash_flow.csv', header=None)
income_statement = pd.read_csv('data/financials/AAPL_income_statement.csv', header=None)
ttm_cash_flow = pd.read_csv('data/financials/AAPL_ttm_cash_flow.csv', header=None)

Holders

In [3]:
insider_roster_holders = pd.read_csv('data/holders/AAPL_insider_roster_holders.csv')
institutional_holders = pd.read_csv('data/holders/AAPL_institutional_holders.csv')
mutualfund_holders = pd.read_csv('data/holders/AAPL_mutualfund_holders.csv')
major_holders = pd.read_csv('data/holders/AAPL_major_holders.csv')

Insider

In [4]:
insider_purchases = pd.read_csv('data/insider/AAPL_insider_purchases.csv')
insider_transactions = pd.read_csv('data/insider/AAPL_insider_transactions.csv')

Price

In [5]:
actions = pd.read_csv('data/price/AAPL_actions.csv')
dividends = pd.read_csv('data/price/AAPL_dividends.csv', index_col=0, parse_dates=True)
ohlc = pd.read_csv('data/price/AAPL_ohlc.csv', index_col=0, parse_dates=True)
options = pd.read_csv('data/price/AAPL_options.csv')
volume = pd.read_csv('data/price/AAPL_volume.csv', index_col=0, parse_dates=True)

/tmp/ipykernel_29729/3647304182.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ohlc = pd.read_csv('data/price/AAPL_ohlc.csv', index_col=0, parse_dates=True)


# Data exploration

Price

In [6]:
ohlc.tail()

,Close,High,Low,Open
Price,,,,
2025-09-24,252.30999755859375,255.74000549316406,251.0399932861328,255.22000122070312
2025-09-25,256.8699951171875,257.1700134277344,251.7100067138672,253.2100067138672
2025-09-26,255.4600067138672,257.6000061035156,253.77999877929688,254.10000610351562
2025-09-29,254.42999267578125,255.0,253.00999450683594,254.55999755859375
2025-09-30,254.5749969482422,255.91900634765625,253.82009887695312,254.8249969482422


In [7]:
ohlc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11293 entries, Ticker to 2025-09-30
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   11292 non-null  object
 1   High    11292 non-null  object
 2   Low     11292 non-null  object
 3   Open    11292 non-null  object
dtypes: object(4)
memory usage: 441.1+ KB


In [8]:
ohlc.dtypes

Close    object
High     object
Low      object
Open     object
dtype: object

In [10]:
numeric_cols = ['Open', 'High', 'Low', 'Close']

for col in numeric_cols:
    ohlc[col] = pd.to_numeric(ohlc[col], errors='coerce')


In [11]:
ohlc.describe(include='all')

,Close,High,Low,Open
count,11291.000000,11291.000000,11291.000000,11291.000000
mean,26.628438,26.896547,26.333672,26.606028
std,55.119763,55.661181,54.513908,55.061007
min,0.037681,0.038109,0.037681,0.038109
25%,0.245132,0.250684,0.240161,0.245127
50%,0.466513,0.473755,0.458864,0.465311
75%,20.886218,21.058807,20.690546,20.888417
max,258.103729,259.179926,256.718662,257.276679


In [12]:
ohlc.memory_usage()

Index    90344
Close    90344
High     90344
Low      90344
Open     90344
dtype: int64

In [13]:
print(len(ohlc['Close'].unique()))

8279


In [14]:
all_time_low = ohlc['Low'].min()
all_time_low_date = ohlc['Low'].idxmin()
print(f'All time low: {all_time_low} on {all_time_low_date}')

all_time_high = ohlc['High'].max()
all_time_high_date = ohlc['High'].idxmax()
print(f'All time high: {all_time_high} on {all_time_high_date}')

All time low: 0.0376811251044273 on 1982-07-08
All time high: 259.1799258751944 on 2024-12-26


In [15]:
ohlc.corr()

,Close,High,Low,Open
Close,1.000000,0.999940,0.999937,0.999868
High,0.999940,1.000000,0.999911,0.999933
Low,0.999937,0.999911,1.000000,0.999940
Open,0.999868,0.999933,0.999940,1.000000


Volume

In [16]:
volume.tail()

,Volume
Date,
2025-09-24,42303700
2025-09-25,55202100
2025-09-26,46076300
2025-09-29,40091100
2025-09-30,13712214


In [17]:
volume.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11291 entries, 1980-12-12 to 2025-09-30
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Volume  11291 non-null  int64
dtypes: int64(1)
memory usage: 176.4 KB


In [18]:
volume.dtypes

Volume    int64
dtype: object

In [20]:
volume.describe(include='all')

,Volume
count,1.129100e+04
mean,3.112278e+08
std,3.337518e+08
min,0.000000e+00
25%,1.076945e+08
50%,1.999520e+08
75%,3.911250e+08
max,7.421641e+09


In [23]:
all_time_low_vol = volume['Volume'].min()
all_time_low_vol_date = volume['Volume'].idxmin()
print(f'All time low volume: {all_time_low_vol} on {all_time_low_date}')

all_time_high_vol = volume['Volume'].max()
all_time_high_vol_date = volume['Volume'].idxmax()
print(f'All time high volume: {all_time_high_vol} on {all_time_high_date}')

All time low volume: 0 on 1982-07-08
All time high volume: 7421640800 on 2024-12-26
